In [1]:
# start docker deamon
# !sudo docker login

In [2]:
# # in the command line. same env
# export GOOGLE_APPLICATION_CREDENTIALS="/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json.json"
# gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

In [1]:
!gcloud auth list
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [4]:
# in the command line. based on:
# https://nextflow.io/blog/2023/nextflow-with-gbatch.html
# gcloud services enable batch.googleapis.com compute.googleapis.com logging.googleapis.com
# gcloud iam service-accounts create eila-batch
# gcloud iam service-accounts list 




# gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/storage.admin"


In [5]:

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \

In [ ]:
# !nextflow -self-update

In [62]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [7]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [8]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/

docker	hai-gcp-genomic-beb9207a40b8.json  nf_data  nf_processes  nf_workflow


In [9]:
import os
os.getcwd()

'/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac'

In [10]:
%%writefile nextflow_vm.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  conda_docker_profile {
    includeConfig 'conf/conda_docker.config'
    executor = 'local'
    }

  docker_profile {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work'
    wave.enabled = true
    fusion.enabled = true
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = false
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }
    
}

params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv'
params.CHROMAP_IDX='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'



Overwriting nextflow_vm.config


In [11]:
%%writefile nextflow_vm_anvil.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  conda_docker_profile {
    includeConfig 'conf/conda_docker.config'
    executor = 'local'
    }

  docker_profile {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://fc-secure-87394b1b-e8d0-4802-9671-70bc3c8b7525/nextflow'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'terra-addd4c49'
    location = 'us-central1'
    batch.spot = false
    batch.serviceAccountEmail='pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com'
  }
}

params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv'
params.CHROMAP_IDX='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'



Overwriting nextflow_vm_anvil.config


In [12]:
%%writefile nextflow_local.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}
profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  conda_docker_profile {
    includeConfig 'conf/conda_docker.config'
    executor = 'local'
    }

  docker_profile {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = false
    google.batch.serviceAccountEmail='pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com'
  }
    
}
params.FASTQS_SPEC_CH = '/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_local.csv'
params.CHROMAP_IDX='/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'



Overwriting nextflow_local.config


In [13]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [2]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !gsutil rm -r gs://eila-nextflow-bucket/work

In [6]:
# !rm .nextflow*

In [7]:
# !ls nextflow.config

In [8]:
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile





# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


In [11]:
!nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile google_batch_profile -resume

N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_atac.nf` [determined_meninsky] DSL2 - revision: c6ceb8eb5e
Nov 01, 2023 8:40:44 PM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv
calling run_seqspec_print
[-        ] process > run_seqspec_print       -
[-        ] process > run_seqspec_modify_atac -
finished run_seqspec_print
start run_seqspec_modify_atac
finish run_seqspec_modify_atac
start genome_fasta_ch download
finished genome_fasta_ch download
start genome_gtf_ch download
finished genome_gtf_ch download
start genome_chromap_idx download
finished genome_chromap_idx download
start run_chromap_map_to_idx

[-        ] process > run_seqspec_print       -
[-        ] process > run_seqspec_modify_atac -
[-        ] process > run_chromap_map_to_idx  -
finished run_seqspec_print
start run_seqspec_modify_atac
finish run_seqspec_

In [10]:
!cat .nextflow.log 

Nov-01 20:38:12.824 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
Nov-01 20:38:12.981 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.0
Nov-01 20:38:13.011 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.2,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.0
Nov-01 20:38:13.027 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Nov-01 20:38:13.029 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Nov-01 20:38:13.033 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Nov-01 20:38:13.051 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Nov-01 20:38:13.079 [main] DEBUG nextflow.config.ConfigBuilde

In [50]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/seqspec/specs/dogmaseq-dig/fastqs

In [51]:
# !nextflow run /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [38]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [39]:
# !cat .command.out

In [40]:
# !nextflow run nf_wf_fg_singlecell.nf -c nextflow.config -with-docker eilalan/env_synapse:latest